### Default Setting 

In [5]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.node_parser import SentenceSplitter 
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
from llama_index import QueryBundle 
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)
import chromadb
import pandas as pd
import json 
import openai
import os
import getpass
import torch

In [6]:
data_path = os.path.join('/workspace/data/')
index_path = os.path.join('/workspace/db/chroma-llama')
config_path = os.path.join('/workspace/config')

In [7]:
model_name='davidkim205/komt-mistral-7b-v1'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True) # , device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
g_device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(g_device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [9]:
generation_config = GenerationConfig(
    temperature=0.8,
    do_sample=True,
    top_p=0.95,
    max_new_tokens=512,
)

### Retrieve 

In [10]:
from llama_index.retrievers import VectorIndexRetriever 
from llama_index.vector_stores import SimpleVectorStore
from llama_index.query_engine import RetrieverQueryEngine 
from llama_index.postprocessor import SimilarityPostprocessor 
from llama_index.postprocessor import SentenceTransformerRerank 
from llama_index.postprocessor import KeywordNodePostprocessor 
from llama_index.postprocessor import SimilarityPostprocessor, CohereRerank
from llama_index.schema import Node, NodeWithScore 

In [11]:
db = chromadb.PersistentClient(path="/workspace/db/chroma-llama")
desc_collection = db.get_or_create_collection("desc")
feature_collection = db.get_or_create_collection("feature")
qualification_collection = db.get_or_create_collection("qualification")

In [18]:
desc_store = ChromaVectorStore(chroma_collection=desc_collection)
feature_store = ChromaVectorStore(chroma_collection=feature_collection)
qualification_store = ChromaVectorStore(chroma_collection=qualification_collection)

In [12]:
desc_collection.count()   # 2887

1023

#### - Embedding Model  - kakaobank 

In [13]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [14]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [15]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [16]:
embedding_service = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [19]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
features_idx = VectorStoreIndex.from_vector_store(feature_store, service_context=embedding_service)
desc_idx = VectorStoreIndex.from_vector_store(desc_store, service_context=embedding_service)
qualification_idx = VectorStoreIndex.from_vector_store(qualification_store, service_context=embedding_service)

#### - Retriever 

In [20]:
retriever = VectorIndexRetriever(
    index = desc_idx,
    service_context=embedding_service,
    similarity_top_k = 10, 
    verbose=True
)

In [21]:
def get_retrieved_nodes(
    retriever, query_str, vector_top_k=10, similarity_cutoff=0.6, reranker_top_n=3, service_context=None, with_reranker=False
):
    # query bundle 생성 
    query_bundle = QueryBundle(query_str)

    # 유사도가 제일 높은 node 추출 
    retrieved_nodes = retriever.retrieve(query_bundle)

    # 전처리  - 유사 점수 기준 Cutoff 
    node_postprocessors = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)
    processed_nodes = node_postprocessors.postprocess_nodes(retrieved_nodes)
    
    if with_reranker:   # 재순위화 
        reranker = SentenceTransformerRerank(
            model='bongsoo/albert-small-kor-cross-encoder-v1',
            top_n=reranker_top_n,
        )
        reranked_nodes = reranker.postprocess_nodes(
            processed_nodes, query_bundle
        )
    return reranked_nodes

In [22]:
with_reranker = True
cutoff = 0.2

queries = ['렌탈 요금 청구 할인', '주유소 할인', 'KT 통신 요금 할인', '아파트 관리비', '청년들을 위한 적금 상품 '] 
query = '렌탈 요금 청구 할인' # queries[0]

In [24]:
import time

start = time.time()
retrieved_nodes = retriever.retrieve(query)
nodes = get_retrieved_nodes(retriever, query, similarity_cutoff=cutoff, service_context=embedding_service, with_reranker=with_reranker)
end = time.time() - start 
print(end)

0.7633793354034424


In [25]:
nodes[0].score

0.65832305

In [26]:
nodes[0].node.relationships['1'].metadata

{'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'}

In [27]:
for key, value in nodes[0].node.relationships.items():
    print(f"key: {key}, value: {value}", end='\n\n')

key: 1, value: node_id='카드_1064' node_type=<ObjectType.DOCUMENT: '4'> metadata={'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'} hash='90392ac21267d9c5d97da6f6c6c1d471c44ff79c0739632924fc94e81f1e6dd5'

key: 2, value: node_id='09bf1380-fd5b-4481-8fbb-ccac80323db1' node_type=<ObjectType.TEXT: '1'> metadata={'category': 'card', 'name': 'LG전자 플러스 하나카드'} hash='d51f79921ab96c6dfb2987685bd78bd4904a27558d504495f6ceb9ab01dfc8c7'

key: 3, value: node_id='6574a0f1-0ea0-4809-beed-7c99b5218b98' node_type=<ObjectType.TEXT: '1'> metadata={} hash='2255a2418f9a9575fff5e8c4ec130a434b07d8d1be99a42019b79a788460cb2a'



In [28]:
def get_info(retrieved_node):
    '''
    id, text, score 반환 
    '''
    id = retrieved_node.node.relationships['1'].node_id
    name = retrieved_node.node.relationships['1'].metadata['name']
    txt = retrieved_node.node.text 
    score = retrieved_node.score 
    return [id, name, txt, score] 

In [29]:
nodes

[NodeWithScore(node=TextNode(id_='634869f8-d3b7-46a0-82dd-7346cdcfd4a7', embedding=None, metadata={'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category', 'name'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='카드_1064', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'}, hash='90392ac21267d9c5d97da6f6c6c1d471c44ff79c0739632924fc94e81f1e6dd5'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='09bf1380-fd5b-4481-8fbb-ccac80323db1', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'card', 'name': 'LG전자 플러스 하나카드'}, hash='d51f79921ab96c6dfb2987685bd78bd4904a27558d504495f6ceb9ab01dfc8c7'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6574a0f1-0ea0-4809-beed-7c99b5218b98', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2255a2418f9a9575fff5e8c4ec130a434b07d8d1be99a42019b79a788460cb2a')}, hash='1221d93222d3aabae38c84ccbe1a536bf41

In [30]:
print([node.text for node in retrieved_nodes])

['렌탈요금 자동이체 시 13,000원 청구할인', '렌탈 자동이체 시 월 8,000원 또는 월 13,000원 할인', '실속만점 생활할인은 기본!&lt이제 렌탈요금 까지 할인 받자!', '청호나이스 렌탈요금 자동이체하고 최대 2만원 할인받자!', '＇SK매직 삼성카드＇는 SK매직 상품 구매시 24·36개월 무이자 할부 혜택이 아닌 렌탈료 할인 혜택이 제공되는 카드입니다', '렌탈료 및 장기할부 이용 시 최대 20,000원 할인!', '바디프랜드 렌탈요금 청구할인과 장기할부 서비스!', '대명아임레디 자동이체 시 최대 19,000원 청구할인', '현대렌탈케어 렌탈요금 할인에, 다양한 생활 혜택까지 누리자!', '7개업종중선택한1개업종가맹점에서카드사용시7%청구할인']


In [31]:
contexts = [node.text for node in nodes]
scores = [node.score for node in nodes]
contexts, scores 

(['렌탈요금 자동이체 시 13,000원 청구할인',
  '바디프랜드 렌탈요금 청구할인과 장기할부 서비스!',
  '청호나이스 렌탈요금 자동이체하고 최대 2만원 할인받자!'],
 [0.65832305, 0.5773951, 0.45793614])

In [32]:
get_info(nodes[0])

['카드_1064', 'NEW 렌탈 플러스 하나카드', '렌탈요금 자동이체 시 13,000원 청구할인', 0.65832305]

In [33]:
info = get_info(nodes[0])
info

['카드_1064', 'NEW 렌탈 플러스 하나카드', '렌탈요금 자동이체 시 13,000원 청구할인', 0.65832305]

In [34]:
product = info[1]
context = f"'{info[1]}'은 {info[2]} 관련 상품입니다."
product, context

('NEW 렌탈 플러스 하나카드', "'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다.")

### Augment 

In [35]:
prompt_template = (
    f"""<s>[INST] 질문: {query} \n
    관련 정보: {context} \n 
    관련 정보를 바탕으로 질문에 답해줘 [/INST] \n"""
)

print(prompt_template)

<s>[INST] 질문: 렌탈 요금 청구 할인 

    관련 정보: 'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다. 
 
    관련 정보를 바탕으로 질문에 답해줘 [/INST] 



### Generation 

In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

In [37]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [38]:
gened = model.generate(
    **tokenizer(
        prompt_template,
        return_tensors='pt',
        return_token_type_ids=False
    ).to('cuda'),
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

<s><s> [INST] 질문: 렌탈 요금 청구 할인 

    관련 정보: 'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다. 
 
    관련 정보를 바탕으로 질문에 답해줘 [/INST] 
1. 렌탈 요금 청구 할인은 NEW 렌탈 플러스 하나카드를 이용하여 렌탈요금 자동이체 시 13,000원이 청구할인되는 혜택을 제공하는 상품입니다.
2. NEW 렌탈 플러스 하나카드는 렌탈 요금 청구 할인 혜택을 받을 수 있는 카드로, 렌탈 자동이체 시 13,000원이 청구할인됩니다.
3. 렌탈 요금 청구 할인은 NEW 렌탈 플러스 하나카드를 이용해야만 가능합니다. 
4. NEW 렌탈 플러스 하나카드를 사용하여 렌탈요금 자동이체 시 13,000원이 청구할인됩니다.</s>


In [39]:
result_str = tokenizer.decode(gened[0])

start_tag = f"[/INST]"
start_index = result_str.find(start_tag)

if start_index != -1:
    result_str = result_str[start_index + len(start_tag):].strip()

In [40]:
print(result_str)

1. 렌탈 요금 청구 할인은 NEW 렌탈 플러스 하나카드를 이용하여 렌탈요금 자동이체 시 13,000원이 청구할인되는 혜택을 제공하는 상품입니다.
2. NEW 렌탈 플러스 하나카드는 렌탈 요금 청구 할인 혜택을 받을 수 있는 카드로, 렌탈 자동이체 시 13,000원이 청구할인됩니다.
3. 렌탈 요금 청구 할인은 NEW 렌탈 플러스 하나카드를 이용해야만 가능합니다. 
4. NEW 렌탈 플러스 하나카드를 사용하여 렌탈요금 자동이체 시 13,000원이 청구할인됩니다.</s>


#### Llama Index response synthesizer 